In [7]:
import numpy as np
import emcee
import pandas as pd
import corner
from scipy.optimize import fsolve, minimize

import time
import multiprocessing
from itertools import product
from functools import partial

%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

In [2]:
def lnlikelihood(theta, f, t, f_err):
    a, a_prime, t_0, t_b, alpha_r, alpha_d, s, sig_0 = theta

    pre_exp = np.logical_not(t > t_0)
    model = np.empty_like(f)
    model[pre_exp] = a
    
    time_term = (t[~pre_exp] - t_0)/t_b
    model[~pre_exp] = a_prime * (time_term)**alpha_r * (1 + (time_term)**(s*alpha_d))**(-2/s)
    
    ln_l = np.sum(np.log(1. / np.sqrt(2*np.pi * (sig_0**2 + f_err**2))) - ((f - model)**2 / (2 * (sig_0**2 + f_err**2))))
    return ln_l

def nll(theta, f, t, f_err):
    return -1*lnlikelihood(theta, f, t, f_err)

#Define priors on parameters  
def lnprior(theta):
    a, a_prime, t_0, t_b, alpha_r, alpha_d, s, sig_0 = theta
    if (-1e8 < t_0 < 1e8 and 0 < alpha_r < 1e8 and 
        0 < alpha_d < 1e8 and 0 < sig_0 < 1e8 and 
        -1e8 < a < 1e8 and  0 < t_b < 1e8 and 
        0 < s < 1e8 and 0 < a_prime < 1e8):
        return 0.0
    return -np.inf

def lnposterior(theta, f, t, f_err):
    lnp = lnprior(theta)
    lnl = lnlikelihood(theta, f, t, f_err)
    if not np.isfinite(lnl):
        return -np.inf
    if not np.isfinite(lnp):
        return -np.inf
    return lnl + lnp 

In [47]:
def sn_lc_mcmc(snname, sigma_sys, sn_dict):
    
    sn_lc_obj = SimSnIa(snname)
    sn_lc_obj.dist_ = sn_dict[snname]['dist']
    sn_lc_obj.z_ = sn_dict[snname]['z']
    sn_lc_obj.mu_ = sn_dict[snname]['mu']
    sn_lc_obj.alpha_r_ = sn_dict[snname]['alpha_r']
    sn_lc_obj.t_b_ = sn_dict[snname]['t_b']
    sn_lc_obj.s_ = sn_dict[snname]['s']
    sn_lc_obj.alpha_d_ = sn_dict[snname]['alpha_d']
    sn_lc_obj.a_prime_ = sn_dict[snname]['a_prime']
    sn_lc_obj.t_exp_ = sn_dict[snname]['t_exp']
    sn_lc_obj.calc_ft(sn_dict[snname]['t_obs'])
    
    sn_lc_obj.calc_noisy_lc(sigma_sys=sigma_sys)
        
    t_rest = sn_lc_obj.t_obs_/(1 + sn_lc_obj.z_)
    f_data = sn_lc_obj.cnts_
    f_unc_data = sn_lc_obj.cnts_unc_

    #initial guess on parameters
    guess_0 = [0, 2*np.max(f_data), 0, 20, 1.5, 1.5, 1, 0]

    # initialize near maximum-likelihood result
    ml_res = minimize(nll, guess_0, method='Powell', # Powell method does not need derivatives
                      args=(f_data, t_rest, f_unc_data))
    ml_guess = ml_res.x
#     print(ml_guess)
    if ml_guess[-1] < 0:
        ml_guess[-1] *= -1

    #number of walkers
    nwalkers = 100
    nfac = [1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2]
    ndim = len(guess_0)

    #initial position of walkers
    pos = [ml_guess + nfac * np.random.randn(ndim) for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, 
                                    args=(f_data, t_rest, f_unc_data))
    nsamples = 2500
    foo = sampler.run_mcmc(pos, nsamples)

    # intermediate file to write out data
    filename = "{}.h5".format(sn_lc_obj.name_)
    backend = emcee.backends.HDFBackend(filename, name='{}'.format(sigma_sys))
    backend.reset(nwalkers, ndim)
    
    # run second burn in
    burn_max_post = sampler.chain[:,-1,:][np.argmax(sampler.lnprobability[-1,:])]
    pos = [burn_max_post + nfac * np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, 
                                    args=(f_data, t_rest, f_unc_data), 
                                    backend=backend)
    nsamples = 500
    foo = sampler.run_mcmc(pos, nsamples)
    
    return sampler

In [45]:
def sim_sn_lcs(snname_list):
    sim_dict = {}
    
    for snname in snname_list:
        sn = SimSnIa(snname)
        sn.draw_dist_in_volume(d_max=200)
        sn.draw_alpha_r()
        sn.draw_rise_time()
        sn.draw_smoothing_parameter()
        sn.draw_mb_deltam15()
        sn.calc_alpha_d()
        sn.calc_a_prime()

        t_obs = np.arange(-40, 30, 1, dtype=float) + np.random.uniform(-0.25/24,0.25/24,size=70)

        sn.calc_ft(t_obs)

        loop_dict = {'dist': sn.dist_, 
                     'z': sn.z_,
                     'mu': sn.mu_,
                     'alpha_r': sn.alpha_r_,
                     't_p': sn.t_p_,
                     't_b': sn.t_b_,
                     's': sn.s_,
                     'alpha_d': sn.alpha_d_,
                     'a_prime': sn.a_prime_,
                     't_exp': sn.t_exp_, 
                     't_obs': t_obs}

        sim_dict['{}'.format(sn.name_)] = loop_dict

    return sim_dict

In [46]:
sim_dict = sim_sn_lcs(['SN2008A', 'SN2008B', 'SN2008C'])

In [43]:
sim_dict
snname = 'SN2008A'
sn = SimSnIa(snname)
sn.dist_ = sim_dict[snname]['dist']
sn.z_ = 72*sn.dist_/2.997942e5
sn.mu_ = 5*np.log10(sn.dist_) + 25
sn.alpha_r_ = sim_dict[snname]['alpha_r']
sn.t_b_ = sim_dict[snname]['t_b']
sn.s_ = sim_dict[snname]['s']
sn.alpha_d_ = sim_dict[snname]['alpha_d']
sn.a_prime_ = sim_dict[snname]['a_prime']
sn.t_exp_ = sim_dict[snname]['t_exp']
sn.calc_ft(sim_dict[snname]['t_obs'])

In [48]:
def same_noise_diff_sn(snname_list, sim_dict, noise):

    pool = multiprocessing.Pool(len(snname_list))
        
    sampler_list = ['sampler{:d}'.format(x) for x in range(len(snname_list))]
    exec("""{} = pool.map(partial(sn_lc_mcmc, sigma_sys=noise, sn_dict=sim_dict), 
                                  snname_list)""".format(', '.join(sampler_list)))

In [50]:
same_noise_diff_sn(['SN2008A', 'SN2008B', 'SN2008C'], sim_dict, 5)

/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/s

In [9]:
sn_pool = multiprocessing.Pool(2)
sn_dict1, sn_dict2 = sn_pool.map(partial(same_sn_diff_noise, nloop=3, noise_list=[3,5,50,316]), 
                                 [1, 2])

sn_dict = dict(sn_dict1, **sn_dict2) #; d4.update(d3)
np.save('sn_dict.npy', sn_dict)

AssertionError: daemonic processes are not allowed to have children

In [5]:
sim_dict = {}

for sim_num in range(3):
    sn = SimSnIa()
    sn.draw_dist_in_volume(d_max=200)
    sn.draw_alpha_r()
    sn.draw_rise_time()
    sn.draw_smoothing_parameter()
    sn.draw_mb_deltam15()
    sn.calc_alpha_d()
    sn.calc_a_prime()

    t_obs = np.arange(-40, 30, 1, dtype=float) + np.random.uniform(-0.25/24,0.25/24,size=70)

    sn.calc_ft(t_obs)

    pool = multiprocessing.Pool(4)
    sampler1, sampler2, sampler3, sampler4 = pool.map(partial(sn_lc_mcmc, sn_lc_obj=sn, loop_num=sim_num), 
                                                      [3,5,50,316])
    
    loop_dict = {'dist': sn.dist_,
                 'alpha_r': sn.alpha_r_,
                 't_p': sn.t_p_,
                 't_b': sn.t_b_,
                 's': sn.s_,
                 'alpha_d': sn.alpha_d_,
                 'a_prime': sn.a_prime_,
                 't_exp': sn.t_exp_}
    
    sim_dict['sim_num{}'.format(sim_num)] = loop_dict


/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher

/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python

In [6]:
sim_dict
np.save('sim_dict.npy', sim_dict)

In [7]:
sim_dict

{'sim_num0': {'dist': 146.8781690698475,
  'alpha_r': 2.4585470074714344,
  't_p': 16.598923309369116,
  't_b': 16.56444887630001,
  's': 2.1630163717377164,
  'alpha_d': 2.4451042367425755,
  'a_prime': 4960.789810046426,
  't_exp': 0},
 'sim_num1': {'dist': 139.2857200563874,
  'alpha_r': 1.0148271366739499,
  't_p': 16.938868807290685,
  't_b': 26.517688503717967,
  's': 1.2058320525640536,
  'alpha_d': 2.7479524192244997,
  'a_prime': 7761.079458597715,
  't_exp': 0},
 'sim_num2': {'dist': 149.8704952400043,
  'alpha_r': 1.5571344647246963,
  't_p': 16.80562161089516,
  't_b': 19.07008384131101,
  's': 2.3120681894360167,
  'alpha_d': 2.3061900788814187,
  'a_prime': 4182.417501048039,
  't_exp': 0}}

In [36]:
t_0_res = np.empty((100,5))
t_b_res = np.empty_like(t_0_res)
alpha_r_res = np.empty_like(t_0_res)
alpha_d_res = np.empty_like(t_0_res)
sig_0_res = np.empty_like(t_0_res)
a_res = np.empty_like(t_0_res)
s_res = np.empty_like(t_0_res)
a_prime_res = np.empty_like(t_0_res)

burnin = 3000

for loop_num in range(100):
    filename = 'noise3_loop{}.h5'.format(loop_num)
    reader = emcee.backends.HDFBackend(filename)
    
    
    samples = reader.get_chain(discard=burnin, flat=True)
    
    a_mc, a_prime_mc, t_0_mc, t_b_mc, alpha_r_mc, alpha_d_mc, s_mc, sig_0_mc = map(lambda v: (v[0], v[1], v[2], v[3], v[4]), 
                                                                               zip(*np.percentile(samples, [2.5, 16, 50, 84, 97.5], axis=0)))

    t_0_res[loop_num] = t_0_mc
    t_b_res[loop_num] = t_b_mc
    alpha_r_res[loop_num] = alpha_r_mc
    alpha_d_res[loop_num] = alpha_d_mc
    sig_0_res[loop_num] = sig_0_mc
    a_res[loop_num] = a_mc
    s_res[loop_num] = s_mc
    a_prime_res[loop_num] = a_prime_mc


In [37]:
t_exp_true = np.empty(100)
t_b_true = np.empty_like(t_exp_true)
alpha_r_true = np.empty_like(t_exp_true)
alpha_d_true = np.empty_like(t_exp_true)
a_true = np.empty_like(t_exp_true)
s_true = np.empty_like(t_exp_true)
a_prime_true = np.empty_like(t_exp_true)

for loop_num in range(100):
    t_exp_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['t_exp']
    t_b_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['t_b']
    alpha_r_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['alpha_r']
    alpha_d_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['alpha_d']
    s_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['s']
    a_prime_true[loop_num] = sim_dict['sim_num{}'.format(loop_num)]['a_prime']

In [41]:
sum(a_prime_true > 10000)

14

In [39]:
t_0_res

array([[-98.68092149, -90.95987635, -69.58186425, -47.84167225,
        -39.75525298],
       [-98.4581047 , -90.45570133, -70.25505347, -48.99611496,
        -40.2117815 ],
       [-98.47870454, -90.04081688, -68.53608397, -47.52565574,
        -39.62405343],
       [-98.15849817, -89.37842136, -67.42660079, -45.58964906,
         48.19223755],
       [-98.32153578, -89.79335439, -66.74223355, -41.89536122,
         68.15370626],
       [-98.3537403 , -89.24375233, -65.05483138, -39.60872369,
         87.55476984],
       [-98.3980672 , -89.48920008, -70.0243627 , -48.76558616,
        -39.91395143],
       [-98.17002305, -89.3262222 , -65.70136481, -42.81298807,
         61.46979773],
       [-98.23347753, -88.28954693, -63.68602138, -39.41077102,
         90.21676692],
       [-98.27485987, -88.36326248, -64.37402032, -40.91148525,
         85.45960154],
       [-98.44398661, -89.91884254, -67.45192751, -44.39875574,
         50.63179003],
       [-98.22407186, -88.18336311, -65.866

In [51]:
def delta_m15_root(alpha_d, t_p=18, alpha_r=2, s=1, dm15=1.1):
    '''Root solver for alpha_d based on Delta m15
    
    Using Eqn. 4 from Zheng & Filippenko (2017), ApJL, 838, 4, it is 
    possible to calculate the ratio of flux from a SN Ia at t = t_peak
    and t = t_peak + 15. If t_p, alpha_r, and s are known, then the 
    ratio of flux should equal Delta m15. The scipy.optimize root 
    finder fsolve is used to solve for the value of alpha_d.
    
    Parameters
    ----------
    alpha_d : float
        Power-law index for the late-time decline of the SN
    
    t_p : float, optional (default=18)
        Time to peak of the SN light curve

    alpha_r : float, optional (default=2)
        Power-law index for initial rise of the SN light curve 

    s : float, optional (default=1)
        Smoothing parameter for the light curve

    dm15 : float, optional (default=1.1)
        Delta m15
    
    Returns
    -------
    alpha_d_root
        The value of alpha_d that results in a SN light curve
        with a 15 day decline rate = Delta m15
    '''
    
    t_b = t_p/((-alpha_r/2)/(alpha_r/2 - alpha_d))**(1/(s*(alpha_d)))
    
    Ltp = (t_p/t_b)**alpha_r * (1 + (t_p/t_b)**(s*alpha_d))**(-2/s)
    Ltp_15 = ((t_p + 15)/t_b)**alpha_r * (1 + ((t_p + 15)/t_b)**(s*alpha_d))**(-2/s)
    
    return 2.5*np.log10(Ltp/Ltp_15) - dm15


class SimSnIa():
    
    def __init__(self, name=None):
        '''initialize the simulated SN
        
        Attributes
        ----------
        name_ : str (default=None)
            Name of the SN object
        '''
        self.name_ = name
    
    def draw_dist_in_volume(self, d_max=100, H_0=72):
        '''simulate SN at a random distance within a fixed volume
        
        Parameters
        ----------
        d_max : int, optional (default=100)
            Maximum distance for the simulated SNe, units in Mpc
        
        H_0 : float, optional (default=72)
            Value of the Hubble constant (in km/s/Mpc) used to convert the 
        distance to the SN to a redshift, z.
        
        Attributes
        ----------
        dist_ : float
            Distance to the SN in Mpc
        
        z_ : float
            Redshift to the SN
        
        mu_ : float
            distance modulus to the SN            
        '''
        
        self.dist_ = np.random.uniform()**(1/3)*d_max
        self.z_ = H_0*self.dist_/2.997942e5
        self.mu_ = 5*np.log10(self.dist_) + 25
    
    def draw_alpha_r(self, alpha_low=1, alpha_high=2.5):
        '''draw random value for early rise power-law index
        
        Select a random value from a flat distribution between 
        alpha_low and alpha_high to determine the power-law index
        for the initial rise of the SN light curve.
        
        Parameters
        ----------
        alpha_low : float, optional (default=1)
            Minimum value for the power-law index of the early rise
        
        alpha_high : float, optional (default=2.5)
            Maximum value for the power-law index of the early rise
        
        Attributes
        ----------
        alpha_r_ : float
            Power-law index for initial rise of the SN light curve          
        '''
        
        self.alpha_r_ = np.random.uniform(alpha_low, alpha_high)
    
    def draw_rise_time(self, mu_rise=18, sig_rise=1):
        '''draw random value for the light curve rise time
        
        Select a random value from a gaussian distribution with 
        mean, mu_rise (default=18), and standard deviation, 
        sig_rise (default=1). The defaults are selected based on the 
        results from Ganeshalingam et al. 2011, MNRAS, 416, 2607 
        which found that the rise time for SNe Ia can be described 
        as ~ N(18.03, 0.0576).
        
        Parameters
        ----------
        mu_rise : float, optional (default=18)
            Mean value for the rise time of SN Ia
        
        sig_rise : float, optional (default=1)
            Standard deviation of the rise time distribution for 
            SNe Ia
        
        Attributes
        ----------
        t_p_ : float
            Time for the light curve to reach peak brightness          
        '''
        
        self.t_p_ = np.random.normal(mu_rise, sig_rise)
        
    def draw_smoothing_parameter(self, mu_s=2, sig_s=0.5):
        '''draw random value for the smoothing parameter
        
        Select a random value from a truncated gaussian distribution 
        with mean, mu_s (default=2), and standard deviation, 
        sig_s (default=0.5). This parameter is not physical, and 
        is largely degenerate with alpha_decline. It is drawn from 
        a guassian distribution while alpha_decline is selected to 
        ensure a physical value of delta m15.
        
        Parameters
        ----------
        mu_s : float, optional (default=2)
            Mean value for the smoothing parameter
        
        sig_s : float, optional (default=0.5)
            Standard deviation of the smoothing parameter
        
        Attributes
        ----------
        s_ : float
            Smoothing parameter for the light curve          
        '''
        s = -1
        while s < 0:
            s = np.random.normal(mu_s, sig_s)
        
        self.s_ = s
    
    def draw_mb_deltam15(self, pkl_file='phillips_kde.pkl'):
        '''Draw random M_b and Delta m15 values
        
        Draw from a KDE estimate based on Burns et al. 2018 to get 
        M_b and Delta m15 for a "normal" SN Ia.
        
        Parameters
        ----------
        pkl_file : str, filename (defaualt='phillips_kde.pkl')
            Pickle file that contains the KDE estimate of the 
            Phillips relation
                
        Attributes
        ----------
        M_b_ : float
            Rest-frame absolute magnitude in the B band at the 
            time of peak brightness
        dm15_ : float
            Delta m15 for the SN
        '''
        with open(pkl_file, 'rb') as file:  
            sn_tuple = pickle.load(file)
        
        kde, phillips_scaler = sn_tuple
        scaled_sample = kde.sample(1)[0]
        
        self.dm15_, self.M_b_= phillips_scaler.inverse_transform(scaled_sample)

        
    def calc_alpha_d(self, alpha_d_guess=2):
        '''Calculate the value of alpha_d based on Delta m15
        
        Parameters
        ----------
        alpha_d_guess : float, optional (default=2)
            Initial guess to solve for the root of the alpha_d eqn
        
        Attributes
        ----------
        alpha_d_ : float
            Power-law index for the late-time decline of the SN
        '''
        if not (hasattr(self, 't_p_') and hasattr(self, 'alpha_r_') and 
                hasattr(self, 's_') and hasattr(self, 'dm15_')):
            self.draw_alpha_r()
            self.draw_rise_time()
            self.draw_smoothing_parameter()
            self.draw_mb_deltam15()

        alpha_d = fsolve(delta_m15_root, alpha_d_guess, 
                         args=(self.t_p_, self.alpha_r_, 
                               self.s_, self.dm15_))

        self.alpha_d_ = float(alpha_d)

    def calc_a_prime(self):
        '''Calculate the value of Aprime
        
        Determine the normalization constant to generate a 
        SN light curve with peak flux equal to the luminosity
        associated with M_b.
        
        Attributes
        ----------
        t_b_ : float
            "break time" for the broken power-law model
        
        a_prime_ : float
            Amplitude for the SN light curve
        '''
        if not (hasattr(self, 'alpha_d_') and hasattr(self, 'mu_')):
            self.draw_dist_in_volume()
            self.calc_alpha_d()
            
        m_peak = self.M_b_ + self.mu_

        f_peak = 10**(0.4*(25-m_peak))
        
        t_b = self.t_p_/((-self.alpha_r_/2)/(self.alpha_r_/2 - self.alpha_d_))**(1/(self.s_*(self.alpha_d_)))
        
        model_peak = ((self.t_p_)/t_b)**self.alpha_r_ * (1 + ((self.t_p_)/t_b)**(self.s_*self.alpha_d_))**(-2/self.s_)

        a_prime = f_peak/model_peak
        
        self.t_b_ = t_b
        self.a_prime_ = a_prime
    
    def calc_ft(self, t_obs, t_exp=0):
        '''Calculate the model flux at input times t_obs
        
        Use Eqn. 4 of Zheng & Filippenko 2017 to determine the 
        flux from the SN at all input times t_obs.
        
        Parameters
        ----------
        t_obs : array-like of shape = [n_obs]
            Times at which to calculate the flux from the SN
        
        t_exp : float, optional (default=0)
            Time of explosion for the SN model
        
        Attributes
        ----------
        t_obs_ : array-like of shape = [n_obs]
            Times at which the SN flux is measured
        
        t_exp_ : float
            SN time of explosion
            
        model_flux : array-like of shape = [n_obs]
            The model flux at all times t_obs, assuming no noise 
            contributes to the signal from the SN
        '''
        if not hasattr(self, 'a_prime_'):
            self.calc_a_prime()
        
        pre_explosion = np.logical_not(t_obs > t_exp)
        
        model_flux = np.empty_like(t_obs)
        model_flux[pre_explosion] = 0
        
        t_rest = t_obs[~pre_explosion]/(1 + self.z_)
        model_flux[~pre_explosion] = self.a_prime_ * (((t_rest - t_exp)/self.t_b_)**self.alpha_r_ * 
                                     (1 + ((t_rest - t_exp)/self.t_b_)**(self.s_*self.alpha_d_))**(-2/self.s_))
        
        self.t_obs_ = t_obs
        self.t_exp_ = t_exp
        self.model_flux_ = model_flux

    def calc_noisy_lc(self, sigma_sys=20):
        '''Calculate SN light curve with systematic and statistical noise
        
        Parameters
        ----------
        sigma_sys : float, optional (default=20)
            Systematic noise term to noisify the light curve. Telescope 
            system is assumed to have a zero-point of 25, such that 
            m = 25 - 2.5*log10(flux). Thus, 
            sigma_sys(5-sigma limiting mag) = 10**(0.4*(25 - m_lim))/5. 
            Default corresponds to a limiting mag of 20.
        
        Attributes
        ----------
        cnts : array-like of shape = [n_obs]
            noisy flux from the SN light curve
        
        cnts_unc : array-like of shape = [n_obs]
            uncertainty on the noisy flux measurements
        '''
        if not hasattr(self, 'model_flux_'):
            self.calc_ft()

        cnts = np.zeros_like(self.t_obs_)
        cnts_unc = np.zeros_like(self.t_obs_)

        pre_explosion = np.logical_not(self.t_obs_ > self.t_exp_)
        cnts[pre_explosion] = np.random.normal(0, sigma_sys, size=sum(pre_explosion))
        cnts_unc[pre_explosion] = np.ones_like(self.t_obs_)[pre_explosion]*sigma_sys

        sn_flux = self.model_flux_[~pre_explosion]
        sn_with_random_noise = sn_flux + np.random.normal(np.zeros_like(sn_flux), np.sqrt(sn_flux))
        sn_with_random_plus_sys = sn_with_random_noise + np.random.normal(0, sigma_sys, size=len(sn_flux))

        # total uncertainty = systematic + Poisson
        sn_uncertainties = np.hypot(np.sqrt(np.maximum(sn_with_random_noise, 
                                                       np.zeros_like(sn_with_random_noise))), 
                                    sigma_sys)

        cnts[~pre_explosion] = sn_with_random_plus_sys
        cnts_unc[~pre_explosion] = sn_uncertainties

        self.cnts_ = cnts
        self.cnts_unc_ = cnts_unc